In [ ]:
from io import BytesIO
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    ImageMessageContent,
    LocationMessageContent
)
import cwa
from PIL import Image
from google.cloud import storage


with open('env.json', encoding='utf-8') as f:
    env = json.load(f)


storage_client = storage.Client.from_service_account_json('service.json')
bucket = storage_client.bucket('myfirstprojectnumberonebucket')

app = Flask(__name__)

configuration = Configuration(access_token=env.get('CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(env.get('CHANNEL_SECRET'))


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    print(event.message.text)  #
    print(event.source.user_id) #
    print(event.timestamp)
    print(event.reply_token)

    # ask = event.message.text
    # if ask == 'hello':
    #     ans = '我很好'
    # elif ask == 'hi':
    #     ans = '您哪位 ?'
    # else:
    #     ans = '我不知道妳再說甚麼'

    ask = event.message.text
    ask_map = {'hello': '我很好', 'hi': '您哪位'}
    #ans = ask_map.get(ask, '我不知道妳再說甚麼')
    ans = ask_map.get(ask)
    if not ans:
        ans = cwa.cwa2(ask, env.get('CWA_KEY'))
        ans = cwa.tostr(ans, '\n') or '無此站'
    
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=ans)]
            )
        )

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_message(event):
    print(event.source.user_id) #
    print(event.timestamp)
    print(event.reply_token)

    print(event.message.latitude)
    print(event.message.longitude)
    site = (event.message.latitude, event.message.longitude)
    ans = cwa.cwa2(site, env.get('CWA_KEY'))
    ans = cwa.tostr(ans, '\n') or '無此站'
    
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=ans)]
            )
        )

@handler.add(MessageEvent, message=ImageMessageContent)
def handle_content_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_blob_api = MessagingApiBlob(api_client)
        message_content = line_bot_blob_api.get_message_content(message_id=event.message.id)
        image = Image.open(BytesIO(message_content))
        text = f'{image.height} X {image.width}'

        # upload image
        blob_name = f'{event.source.user_id}_{event.message.id}'
        blob = bucket.blob(blob_name)
        blob.upload_from_string(message_content, content_type='image/jpeg')

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    TextMessage(text=text)
                ]
            )
        )


if __name__ == "__main__":
    app.run(port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit


U091e03d020e37cce02ba9fd2f03377ec
1763178236296
5a3df8a6048e4abdbaa0dff363c0c94e
25.050793
121.569837
sitemaps built


127.0.0.1 - - [15/Nov/2025 11:44:00] "POST /callback HTTP/1.1" 200 -


In [8]:
import time

t = 1762579212668
time.localtime(t/1000)

time.struct_time(tm_year=2025, tm_mon=11, tm_mday=8, tm_hour=13, tm_min=20, tm_sec=12, tm_wday=5, tm_yday=312, tm_isdst=0)

In [ ]:
!pip install pillow

In [22]:
from PIL import Image

image = Image.open('test.jpg')
image.size

(960, 1706)

In [23]:
image.height

1706

In [26]:
image.width

960

In [ ]:
960 x 1706

In [ ]:
!pip install google-cloud-storage

In [1]:
from google.cloud import storage

storage_client = storage.Client.from_service_account_json('service.json')

In [3]:
bucket = storage_client.bucket('myfirstprojectnumberonebucket')

In [4]:
blob = bucket.blob('xxx.jpg')
blob.upload_from_filename('donav.jpg')

In [35]:
bucket.blob('abc.jpg').upload_from_filename('donav.jpg')

In [5]:
help(blob.upload_from_string)

Help on method upload_from_string in module google.cloud.storage.blob:

upload_from_string(data, content_type='text/plain', client=None, predefined_acl=None, if_generation_match=None, if_generation_not_match=None, if_metageneration_match=None, if_metageneration_not_match=None, timeout=60, checksum='auto', retry=<google.api_core.retry.retry_unary.Retry object at 0x000001BDCCD470E0>, crc32c_checksum_value=None) method of google.cloud.storage.blob.Blob instance
    Upload contents of this blob from the provided string.

    .. note::
       The effect of uploading to an existing blob depends on the
       "versioning" and "lifecycle" policies defined on the blob's
       bucket.  In the absence of those policies, upload will
       overwrite any existing contents.

       See the [`object versioning`](https://cloud.google.com/storage/docs/object-versioning)
       and [`lifecycle`](https://cloud.google.com/storage/docs/lifecycle)
       API documents for details.

    If :attr:`user_proje